In [41]:
import pandas as pd

In [42]:
df  = pd.read_csv('flickr8k/captions.txt')
print(df.size)
print(df.head()) 

80910
                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  


In [43]:
#see for each image there are 5 text lines 1->a  1->b  .. 1->e   let group them 
#now can groupy
image_captions = df.groupby("image")["caption"].apply(list).to_dict()
# df.groupby("image") – groups the DataFrame rows by the "image" column
# ["caption"] – selects the "caption" column within each group.
# .apply(list) – converts all captions in each group into a list.
# .to_dict() – converts the grouped result into a dictionary.

In [44]:
first_key = list(image_captions.keys())[0]
print(first_key)
print(image_captions[first_key])
print('10815824_2997e03d76.jpg')
print(image_captions['10815824_2997e03d76.jpg'])

1000268201_693b08cb0e.jpg
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
10815824_2997e03d76.jpg
['A blonde horse and a blonde girl in a black sweatshirt are staring at a fire in a barrel .', 'A girl and her horse stand by a fire .', "A girl holding a horse 's lead behind a fire .", 'A man , and girl and two horses are near a contained fire .', 'Two people and two horses watching a fire .']


In [45]:
type(image_captions)

dict

In [46]:
from collections import defaultdict
# """
#  Summary
# Feature	dict	defaultdict
# Handles missing keys	❌ No (gives error)	✅ Yes (auto-creates default)
# Needs key check	✅ Yes	❌ No
# Cleaner code	❌ Longer	✅ Shorter
# Use when	You want full control	You want convenience
# """
import nltk
import os
from nltk import word_tokenize, pos_tag



In [47]:
image_keywords = defaultdict(set)  #it is a dict having key as image 1.jpg and value as set of noun words
# Creates an empty defaultdict where each value is a set.
# 💡 Why: We’ll store, for each image, the set of all nouns found in its captions.
# ✨ Benefit: No need to check if the key exists before adding to it.

for image, captions in image_captions.items():
    for caption in captions:
        words = word_tokenize(caption.lower())
        nouns = [word for word, tag in pos_tag(words) if tag.startswith('NN')]
        # nouns = []    or this for loop
        # for word, tag in pos_tag(words):
            #if tag.startswith('NN'):
                #     nouns.append(word)
        image_keywords[image].update(nouns)
   




In [48]:
first_key_image = list(image_captions.keys())[0]
print("Keywords for", first_key_image, ":", image_keywords[first_key_image])


Keywords for 1000268201_693b08cb0e.jpg : {'dress', 'child', 'set', 'entry', 'cabin', 'building', 'way', 'girl', 'pink', 'stairs', 'playhouse'}


In [49]:
# we do like any got keywords same >0 then simi else disimi
import random # pick a random positive and a random negative example for each anchor.

In [50]:
#now will iterate over all image get its keyworks from image_captions to compare 
triplets = []
image_list = list(image_keywords.keys())
#now iterate and take respective keywords for each image
for anchor in image_list:
    anchor_keywords = image_keywords[anchor]
    
    #now to find positive at least one same keyword
    positives = [img for img in image_list if img!=anchor and  len(image_keywords[img] & anchor_keywords)>0]
    negatives = [img for img in image_list if img!=anchor and len(image_keywords[img] & anchor_keywords)==0]
    
    if positives and negatives:
        pos = random.choice(positives)
        neg = random.choice(negatives)
        triplets.append((anchor,pos,neg))

print("Total triplets created:", len(triplets))
print("Example triplet:", triplets[0])

#see now i caan access keywords also by these keys(image name in triplets)

Total triplets created: 8091
Example triplet: ('1000268201_693b08cb0e.jpg', '3308997740_91765ecdcc.jpg', '2461631708_decc5b8c87.jpg')


 Build tf.data.Dataset of Raw Paths + Captions

In [51]:
def get_triplet_paths_and_captions(triplets, image_captions):
    def generator():
        for a, p, n in triplets:
            yield (
                f"flickr8k/images/{a}", " ".join(image_keywords[a]),   # Anchor keywords as text
                f"flickr8k/images/{p}", " ".join(image_keywords[p]),   # Positive
                f"flickr8k/images/{n}", " ".join(image_keywords[n]),   # Negative
            )
    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec([], tf.string),  # anchor image path
            tf.TensorSpec([], tf.string),  # anchor keyword text
            tf.TensorSpec([], tf.string),  # pos image path
            tf.TensorSpec([], tf.string),  # pos keyword text
            tf.TensorSpec([], tf.string),  # neg image path
            tf.TensorSpec([], tf.string),  # neg keyword text
        )
    )


 Define Multimodal Model + Triplet Loss

In [52]:
def preprocess_image(path, size=(224, 224)):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, size)
    return tf.keras.applications.resnet50.preprocess_input(img)


Format & Train

In [ ]:
def preprocess_triplet(a_path, a_txt, p_path, p_txt, n_path, n_txt):
    return (
        preprocess_image(a_path), a_txt,
        preprocess_image(p_path), p_txt,
        preprocess_image(n_path), n_txt
    )


In [ ]:
def build_dataset(triplets, image_captions, batch_size=16):
    raw_ds = get_triplet_paths_and_captions(triplets, image_captions)
    ds = (
        raw_ds
        .map(preprocess_triplet, num_parallel_calls=tf.data.AUTOTUNE)
        .shuffle(1000)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return ds


In [55]:
def build_image_branch():
    base_model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", pooling="avg", input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze ResNet
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    return tf.keras.Model(inputs, x, name="ImageBranch")


In [56]:
import tensorflow_hub as hub

def build_text_branch():
    use = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False)
    inputs = tf.keras.Input(shape=(), dtype=tf.string)
    x = use(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    return tf.keras.Model(inputs, x, name="TextBranch")


In [57]:
def build_fusion_model():
    inputs = tf.keras.Input(shape=(512,))  # 256 img + 256 text
    x = tf.keras.layers.Dense(128)(inputs)
    x = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=-1))(x)
    return tf.keras.Model(inputs, x, name="FusionHead")


In [58]:
def triplet_loss(anchor, positive, negative, margin=0.3):
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)


In [59]:
# 1. Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# 2. Training Loop
EPOCHS = 5  # Set to higher later

for epoch in range(EPOCHS):
    total_loss = 0
    num_batches = 0

    for batch in ds:
        anchor_img, pos_img, neg_img, anchor_text, pos_text, neg_text = batch

        with tf.GradientTape() as tape:
            # Step 1: Get image embeddings
            anchor_img_emb = image_model(anchor_img, training=True)
            pos_img_emb    = image_model(pos_img, training=True)
            neg_img_emb    = image_model(neg_img, training=True)

            # Step 2: Get text embeddings
            anchor_txt_emb = text_model(anchor_text, training=True)
            pos_txt_emb    = text_model(pos_text, training=True)
            neg_txt_emb    = text_model(neg_text, training=True)

            # Step 3: Fuse image + text
            a_fused = fusion_model(tf.concat([anchor_img_emb, anchor_txt_emb], axis=1), training=True)
            p_fused = fusion_model(tf.concat([pos_img_emb, pos_txt_emb], axis=1), training=True)
            n_fused = fusion_model(tf.concat([neg_img_emb, neg_txt_emb], axis=1), training=True)

            # Step 4: Compute loss
            loss = triplet_loss(a_fused, p_fused, n_fused)

        # Step 5: Backpropagation
        grads = tape.gradient(loss,
                              image_model.trainable_weights +
                              text_model.trainable_weights +
                              fusion_model.trainable_weights)
        optimizer.apply_gradients(zip(grads,
                                      image_model.trainable_weights +
                                      text_model.trainable_weights +
                                      fusion_model.trainable_weights))

        total_loss += loss.numpy()
        num_batches += 1

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/num_batches:.4f}")


2025-07-06 16:41:57.580850: W tensorflow/core/framework/op_kernel.cc:1833] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported
2025-07-06 16:41:57.582250: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Exception encountered when calling Functional.call().

[1m{{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to float is not supported [Op:Cast] name: [0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(16,), dtype=string)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
# Save each model to use later
image_model.save("image_model.h5")
text_model.save("text_model.h5")
fusion_model.save("fusion_model.h5")
print("Models saved.")


In [ ]:
from tensorflow.keras.models import load_model

image_model = load_model("image_model.h5")
text_model  = load_model("text_model.h5")
fusion_model = load_model("fusion_model.h5")
print("Models loaded.")


In [ ]:
def get_combined_embedding(image_path, caption):
    # Image preprocessing
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)

    img_emb = image_model.predict(img)
    txt_emb = text_model.predict([caption])  # shape: (1, 256)

    fused_input = tf.concat([img_emb, txt_emb], axis=1)  # shape: (1, 512)
    fused_emb = fusion_model(fused_input)  # shape: (1, 128)
    return fused_emb.numpy()[0]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_images(query_image, query_caption, image_caption_dict, top_k=5):
    query_emb = get_combined_embedding(query_image, query_caption)

    results = []
    for img_path, caption in image_caption_dict.items():
        emb = get_combined_embedding(img_path, caption)
        score = cosine_similarity([query_emb], [emb])[0][0]
        results.append((img_path, score))

    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_k]


In [ ]:
# Example query
query_path = "flickr8k/images/123456.jpg"
query_caption = "A child is playing with a ball"

top_results = recommend_images(query_path, query_caption, image_captions, top_k=5)

for img, score in top_results:
    print(f"{img} | Score: {score:.3f}")
